# [1661. Average Time of Process per Machine](https://leetcode.com/problems/average-time-of-process-per-machine?envType=study-plan-v2&envId=top-sql-50)

**Type:** Easy <br>
**Topics:** Databases <br>
**Companies:** Google, Amazon, Bloomberg, Meta, Adobe, Microsoft, Apple, Uber, Infosys, Yahoo
<hr>

Table: `Activity`
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process at the given timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.
It is guaranteed that each (machine_id, process_id) pair has a 'start' and 'end' timestamp.
```

There is a factory website that has several machines each running the **same number of processes**. Write a solution to find the **average time** each machine takes to complete a process.

The time to complete a process is the `'end' timestamp` minus the `'start' timestamp`. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the `machine_id` along with the average time as `processing_time`, which should be **rounded to 3 decimal places**.

Return the result table in **any order**.

The result format is in the following example.
<hr>

### Examples
- **Example 1:**
    - **Input:** 
        ```
        Activity table:
        +------------+------------+---------------+-----------+
        | machine_id | process_id | activity_type | timestamp |
        +------------+------------+---------------+-----------+
        | 0          | 0          | start         | 0.712     |
        | 0          | 0          | end           | 1.520     |
        | 0          | 1          | start         | 3.140     |
        | 0          | 1          | end           | 4.120     |
        | 1          | 0          | start         | 0.550     |
        | 1          | 0          | end           | 1.550     |
        | 1          | 1          | start         | 0.430     |
        | 1          | 1          | end           | 1.420     |
        | 2          | 0          | start         | 4.100     |
        | 2          | 0          | end           | 4.512     |
        | 2          | 1          | start         | 2.500     |
        | 2          | 1          | end           | 5.000     |
        +------------+------------+---------------+-----------+
        ```
    - **Output:** 
        ```
        +------------+-----------------+
        | machine_id | processing_time |
        +------------+-----------------+
        | 0          | 0.894           |
        | 1          | 0.995           |
        | 2          | 1.456           |
        +------------+-----------------+
        ```
    - **Explanation:** <br>
    There are 3 machines running 2 processes each. <br>
    Machine 0's average time is ((1.520 - 0.712) + (4.120 - 3.140)) / 2 = 0.894 <br>
    Machine 1's average time is ((1.550 - 0.550) + (1.420 - 0.430)) / 2 = 0.995 <br>
    Machine 2's average time is ((4.512 - 4.100) + (5.000 - 2.500)) / 2 = 1.456
<hr>

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, round
import pyspark

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/27 19:33:02 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/27 19:33:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 19:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_Activity = spark.read.option('header', True).option('delimiter', ',').csv('Activity.csv')
pandas_Activity = pd.read_csv('Activity.csv')

spark_Activity.show()
pandas_Activity.head(100)

+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|          end|     1.55|
|         1|         1|        start|     0.43|
|         1|         1|          end|     1.42|
|         2|         0|        start|      4.1|
|         2|         0|          end|    4.512|
|         2|         1|        start|      2.5|
|         2|         1|          end|        5|
+----------+----------+-------------+---------+



,machine_id,process_id,activity_type,timestamp
0,0,0,start,0.712
1,0,0,end,1.520
2,0,1,start,3.140
3,0,1,end,4.120
4,1,0,start,0.550
5,1,0,end,1.550
6,1,1,start,0.430
7,1,1,end,1.420
8,2,0,start,4.100
9,2,0,end,4.512


In [3]:
spark_Activity.printSchema()

root
 |-- machine_id: string (nullable = true)
 |-- process_id: string (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [4]:
pandas_Activity.describe()

,machine_id,process_id,timestamp
count,12.000000,12.000000,12.000000
mean,1.000000,0.500000,2.462833
std,0.852803,0.522233,1.656645
min,0.000000,0.000000,0.430000
25%,0.000000,0.000000,1.243000
50%,1.000000,0.500000,2.025000
75%,2.000000,1.000000,4.105000
max,2.000000,1.000000,5.000000


### SQL

In [5]:
def find_Average_Time(activity: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    # Step 1: Create a temporary SQL view from the DataFrame 'activity'
    # This step allows us to run SQL queries on the DataFrame just like a SQL table.
    activity.createOrReplaceTempView('Activity')

    # Step 2: Write the SQL query to calculate the average processing time for each machine
    sqlQuery = '''
    SELECT 
        a.machine_id,  -- Select the machine_id column from the 'a' table (start activities)
        
        -- Calculate the average time difference between start and end activities for each machine
        ROUND(AVG(b.timestamp - a.timestamp), 3) AS processing_time  -- Calculate the time difference between start and end for each activity pair and compute the average for each machine. Round the result to 3 decimal places.
    
    FROM Activity a  -- Alias 'a' for the start activities in the Activity table.
    
    -- Join the 'a' (start) table with 'b' (end) table based on the following conditions:
    JOIN Activity b ON
        a.machine_id = b.machine_id  -- Ensure the machine_id matches between the start and end activities
        AND a.process_id = b.process_id  -- Ensure the process_id matches between the start and end activities
        AND a.activity_type = 'start'  -- Only consider rows where the activity_type in 'a' is 'start'
        AND b.activity_type = 'end'  -- Only consider rows where the activity_type in 'b' is 'end'
    
    GROUP BY a.machine_id  -- Group by machine_id, so the average is computed for each machine.
    '''

    # Step 3: Execute the SQL query using Spark SQL and store the result in 'output'
    # The query calculates the average processing time per machine by joining start and end activities.
    output = spark.sql(sqlQuery)  # Execute the SQL query on the 'Activity' DataFrame.

    # Step 4: Return the resulting DataFrame with the processing time for each machine
    return output

output = find_Average_Time(activity = spark_Activity)
output.show(100)

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|         0|          0.894|
|         1|          0.995|
|         2|          1.456|
+----------+---------------+



### PySpark

In [6]:
def find_Average_Time(activity: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    # Step 1: Define the join condition
    # We are joining the 'start' and 'end' activities on machine_id, process_id, and activity_type
    condition = (
        (col('a.machine_id') == col('b.machine_id')) &  # Match machine_id between 'start' and 'end' activities
        (col('a.process_id') == col('b.process_id')) &  # Match process_id between 'start' and 'end' activities
        (col('a.activity_type') == 'start') &           # Ensure 'a' refers to 'start' activity type
        (col('b.activity_type') == 'end')               # Ensure 'b' refers to 'end' activity type
    )
    
    # Step 2: Perform the join on the activity DataFrame with itself using the defined condition
    output = activity.alias('a').join(
        activity.alias('b'),  # Join 'a' (start) with 'b' (end) on the condition
        on = condition, 
        how = 'inner'  # Use inner join to only include rows with matching start and end activities
    )\
    .select(  # Select relevant columns from both 'a' (start) and 'b' (end) activities
        col('a.machine_id'),
        col('a.timestamp').alias('start_timestamp'),  # Rename 'a.timestamp' to 'start_timestamp'
        col('b.timestamp').alias('end_timestamp')     # Rename 'b.timestamp' to 'end_timestamp'
     )\
     .groupby(
         col('a.machine_id') # Group the data by machine_id
     )\
     .agg(
         # Calculate the average processing time by subtracting start and end timestamps
         round(avg((col('end_timestamp') - col('start_timestamp'))), 3).alias('processing_time')
     )
    
    # Step 3: Return the result with the machine_id and its corresponding average processing time
    return output
    
output = find_Average_Time(activity = spark_Activity)
output.show(100)

+----------+---------------+
|machine_id|processing_time|
+----------+---------------+
|         0|          0.894|
|         1|          0.995|
|         2|          1.456|
+----------+---------------+



In [7]:
def find_Average_Time(activity: pd.DataFrame) -> pd.DataFrame:
    # Step 1: Filter 'start' and 'end' activities
    a = activity[activity.activity_type == 'start'].copy()  # DataFrame for 'start' activities
    b = activity[activity.activity_type == 'end'].copy()    # DataFrame for 'end' activities
        
    # Step 2: Merge the 'start' and 'end' activities based on 'machine_id' and 'process_id'
    output = a.merge(b, on = ['machine_id', 'process_id'], how = 'inner', suffixes = ['_start', '_end'])
    
    # Step 3: Calculate the time difference between 'start' and 'end' timestamps
    output['timeDiff'] = (output.timestamp_end - output.timestamp_start) 
    
    # Step 4: Select necessary columns and group by 'machine_id' to calculate the average processing time
    output = output[['machine_id', 'timeDiff']]
    
    # Group by 'machine_id' and calculate the mean of 'timeDiff'
    output = output.groupby('machine_id', as_index = False).agg({'timeDiff': 'mean'})
    
    # Step 5: Round the processing time to 3 decimal places
    output['processing_time'] = output['timeDiff'].round(3)
    
    # Drop the intermediate 'timeDiff' column
    output.drop(columns=['timeDiff'], inplace = True)
    
    return output
        
output = find_Average_Time(activity = pandas_Activity)
output.head(100)

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456
